# dependencies

In [ ]:
!pip install torch torchvision diffusers accelerate huggingface_hub datasets

# Import libraries

In [ ]:
import os
import torch
from PIL import Image
from diffusers import StableDiffusionPipeline
from huggingface_hub import login
from accelerate import Accelerator
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Login with Hugging-face

In [ ]:
# Step 1: Authenticate with Hugging Face
login('')  # Replace with your Hugging Face access token

# Set up the environment

In [ ]:
# Step 2: Set up the environment
accelerator = Accelerator()
device = accelerator.device  # Automatically uses GPU if available

# Load Model

In [ ]:
# Step 3: Load the Stable Diffusion model
model_id = "SG161222/Realistic_Vision_V2.0"  # Use the specified model
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to(device)

# Disable the safety checker (optional, use with caution)
def dummy_safety_checker(images, **kwargs):
    return images, [False] * len(images)
pipe.safety_checker = dummy_safety_checker

# Prepare Dataset

In [ ]:
# Step 4: Prepare the dataset
image_dir = "/kaggle/input/private/images"  # Path to your images
prompt_dir = "/kaggle/input/private/prompts"  # Path to your prompts

# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, image_dir, prompt_dir):
        self.image_dir = image_dir
        self.prompt_dir = prompt_dir
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.prompts = [open(os.path.join(prompt_dir, f"{os.path.splitext(f)[0]}.txt")).read().strip() for f in self.image_filenames]
        self.transform = transforms.Compose([
            transforms.Resize((768, 768)),  # Resize images to 768x768
            transforms.ToTensor(),  # Convert to tensor
        ])
    
    def __len__(self):
        return len(self.image_filenames)
    
    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)  # Apply transformations
        prompt = self.prompts[idx]
        return image, prompt

# Load Dataset

In [ ]:
# Load the dataset
dataset = CustomDataset(image_dir, prompt_dir)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Create an empty folder named 'result_data'
os.makedirs('result_data', exist_ok=True)

# Fine Tuned

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for i, (image, prompt) in enumerate(dataloader):
        # Move image to device
        image = image.to(device)

        # Generate images using the prompt
        generated_image = pipe(prompt[0], num_inference_steps=50, guidance_scale=7.5).images[0]  # Adjust parameters
        
        # Save the generated image
        generated_image.save(f"result_data/generated_image_epoch{epoch + 1}_img{i + 1}.png")

# Save Model

In [ ]:
# Save the fine-tuned model
model_save_path = "fine_tuned_model"
os.makedirs(model_save_path, exist_ok=True)
pipe.save_pretrained(model_save_path)
tokenizer_save_path = os.path.join(model_save_path, "tokenizer")
os.makedirs(tokenizer_save_path, exist_ok=True)
pipe.tokenizer.save_pretrained(tokenizer_save_path)

# Make Directory

In [ ]:
mkdir result

# Upload Model on Hugging

In [ ]:
from huggingface_hub import HfApi, HfFolder, login

# Authenticate with Hugging Face
token = ''  # Replace with your new token with write permission
login(token)

# Initialize the Hugging Face API
api = HfApi()

# Use the existing repository
model_repo = "majid230/Realistic_Vision_V2.0"

# Path where your model is saved
model_save_path = "fine_tuned_model"  # Replace with your actual path

# Upload the model to Hugging Face
api.upload_folder(
    folder_path=model_save_path,
    repo_id=model_repo,
    token=token,
    commit_message="Upload fine-tuned model"
)

print(f"Model uploaded to {model_repo}")